In [1]:
import pandas as pd
import math
import matplotlib as mpl
#mpl.use('pdf')
import matplotlib.pyplot as plt
import numpy as np
from collections import OrderedDict
import matplotlib
import seaborn as sns
#Red
red1= sns.color_palette("Reds_d",1)
red2= sns.color_palette("Reds_d",2)
red3= sns.color_palette("Reds_d",3)
red4= sns.color_palette("Reds_d",4)
red5= sns.color_palette("Reds_d",5)
red6= sns.color_palette("Reds_d",6)
red7= sns.color_palette("Reds_d",7)
red8= sns.color_palette("Reds_d",8)

#YellowGreen
yg1= sns.color_palette("YlGn_d",1)
yg2= sns.color_palette("YlGn_d",2)
yg3= sns.color_palette("YlGn_d",3)
yg4= sns.color_palette("YlGn_d",4)
yg5= sns.color_palette("YlGn_d",5)
yg7= sns.color_palette("YlGn_d",7)

#GreenBlue
gb1= sns.color_palette("GnBu_d",1)
gb2= sns.color_palette("GnBu_d",2)
gb3= sns.color_palette("GnBu_d",3)
gb4= sns.color_palette("GnBu_d",4)
gb5= sns.color_palette("GnBu_d",5)
gb6= sns.color_palette("GnBu_d",6)
gb7= sns.color_palette("GnBu_d",7)
gb8= sns.color_palette("GnBu_d",8)

cp2 = list(map(lambda x: sns.desaturate(x,0.9),[red7[2],gb7[4]]))
cp2v1 = list(map(lambda x: sns.desaturate(x,0.9),[red7[2],yg7[0]]))
cp3 = list(map(lambda x: sns.desaturate(x,0.9),[yg7[0],gb7[4],red7[2]]))
#cp4 = list(map(lambda x: sns.desaturate(x,0.9),red1+gb2+yg1))
cp2_2 = list(map(lambda x: sns.desaturate(x,0.9),[red7[0],red7[3],gb7[4],gb7[6]]))
cp_total_spectrum = list(map(lambda x: sns.desaturate(x,0.9),gb7 + yg7 + red7))

In [2]:
sns.set_palette(cp2)
sns.color_palette()

[(0.8807358708189157, 0.32277047289504046, 0.2744659746251443),
 (0.2649160579264386, 0.5270152505446624, 0.6269460463924131)]

In [3]:
sns.set_palette(cp2v1)
sns.color_palette()

[(0.8807358708189157, 0.32277047289504046, 0.2744659746251443),
 (0.6120315263360248, 0.8081330257593234, 0.5357962322183777)]

In [4]:
sns.set_palette(cp3)
sns.color_palette()

[(0.6120315263360248, 0.8081330257593234, 0.5357962322183777),
 (0.2649160579264386, 0.5270152505446624, 0.6269460463924131),
 (0.8807358708189157, 0.32277047289504046, 0.2744659746251443)]

In [5]:
cpbasic = list(map(lambda x: sns.desaturate(x,0.9),['g','r','b','orange','grey']))
sns.set_palette(cpbasic)
sns.color_palette()

[(0.025000000000000022, 0.475, 0.025000000000000022),
 (0.95, 0.050000000000000044, 0.050000000000000044),
 (0.050000000000000044, 0.050000000000000044, 0.95),
 (0.95, 0.6323529411764707, 0.050000000000000044),
 (0.5019607843137255, 0.5019607843137255, 0.5019607843137255)]

In [21]:
reps = 5
implementations = ['cuSPARSE_csr', 'cuSPARSE_hyb', 'cuCSR5']
#['OMP_csr-40', 'cuSPARSE_csr', 'cuSPARSE_hyb', 'cuSPARSE_bsr-2', 'cuSPARSE_bsr-4', 'cuSPARSE_bsr-16', 'cuSPARSE_bsr-64', 'cuCSR5']
dtypes=['double','float']

for dtype in dtypes:
    fname = "silver1-TeslaV100_%s_run_test_matrices.csv" % dtype
    inputdata = pd.read_csv('./silver1_build/%s' % fname, names = ['filename','implementation','m','n','nz','time','gflops'])
#inputdata
#bybench = inputdata.groupby(['filename','implementation','m','n','nz'])
#for desc, frame in bybench:
    #print(desc)
#    if frame['time'].count() < reps:
#        #print(frame.head(1))
#        for i in range(0,reps-frame['time'].count()):
#            inputdata = inputdata.append(frame.head(1), ignore_index = True)  
#inputdata.groupby(['filename','implementation','m','n','nz'])['time'].count()
    in_list = []
    for name in inputdata.filename.unique():
        for imp in implementations:
            df_part = inputdata[ (inputdata['filename']==name) & (inputdata['implementation']==imp)]
            bench_iter = df_part.time.count() 
            if bench_iter < reps:
                for i in range(0,reps-bench_iter):
                    in_list.append([name, imp, 'Error', 'in', 'run', 'no', 'results'])
            df_part = df_part.tail(reps) # Get the last 'reps' benchmarks
            #print(df_part)
            for index, row in df_part.iterrows():
                #print(row)
                in_list.append(row.tolist())
    #print (in_list)
    df_out = pd.DataFrame(in_list, columns = ['filename','implementation','m','n','nz','time','gflops'])         
    outputdata = df_out.sort_values(['filename', 'implementation'])
    outputdata.to_csv('./Results/sorted_cleaned_%s' % fname, index = False)
